In [33]:
import pandas as pd
import numpy as np
import math
import plotly.express as px
import copy
from sklearn.model_selection import train_test_split
epsilon = 0.000000001
C = 1

In [2]:
dataset = pd.read_excel("slides.xlsx")

In [3]:
dataset

,ex,Alt,Bar,Fri,Hun,Pat,Price,Rain,Res,Type,Est,Goal
0,x1,Yes,No,No,Yes,Some,$$$,No,Yes,French,"""0 - 10""",1
1,x2,Yes,No,No,Yes,Full,$,No,No,Thai,"""30 - 60""",0
2,x3,No,Yes,No,No,Some,$,No,No,Burger,"""0 - 10""",1
3,x4,Yes,No,Yes,Yes,Full,$,Yes,No,Thai,"""10 - 30""",1
4,x5,Yes,No,Yes,No,Full,$$$,No,Yes,French,"""> 60""",0
5,x6,No,Yes,No,Yes,Some,$$,Yes,Yes,Italian,"""0 - 10""",1
6,x7,No,Yes,No,No,None,$,Yes,No,Burger,"""0 - 10""",0
7,x8,No,No,No,Yes,Some,$$,Yes,Yes,Thai,"""0 - 10""",1
8,x9,No,Yes,Yes,No,Full,$,Yes,No,Burger,"""> 60""",0
9,x10,Yes,Yes,Yes,Yes,Full,$$$,No,Yes,Italian,"""10 - 30""",0


In [4]:
class node ():
    def __init__(self, gainInfo = None , label  = None, feature = None, type = None , value = None,) :
        self.children = []
        self.feature = feature
        self.type = type
        self.gainInfo = gainInfo
        self.label = label
        self.value = value

In [5]:
def MakeUnique(dataFrame , columnName):
    #print(dataFrame[columnName])
    PossibleValues = pd.unique(dataFrame[columnName])
    #print(PossibleValues)
    indexed = {}
    for count , value in enumerate(PossibleValues):
        indexed[value] = count
    for i in range(len( dataFrame[columnName])):
        dataFrame.loc[i , columnName] =  indexed[dataFrame[columnName][i]] 

In [34]:
def MakeUniqueInt (dataFrame , columnName):
    maxvalue = np.max(dataFrame[columnName])
    minvalue = np.min(dataFrame[columnName])
    lenghtEachSlice = (maxvalue - minvalue + 1 ) / ( len(dataFrame[columnName]) / C)
    indexed = {}
    for count , value in enumerate( range(len(dataFrame[columnName] / lenghtEachSlice)) ):
        indexed[(value * lenghtEachSlice + minvalue , (value + 1) * lenghtEachSlice + minvalue)] = count
    for i in range(len(dataFrame[columnName])):
        temp = dataFrame.loc[i , columnName]
        flag = False
        for j in indexed :
            if temp >= j[0] and  temp < j[1]:
                flag = True
                dataFrame.loc[i , columnName] = indexed[j]
        if flag == False:
            if temp < indexed[0][0] :
                dataFrame.loc[i , columnName] = -1
            else:
                dataFrame.loc[i , columnName] = len(indexed) + 1

In [7]:
def plurality_value(example):
    counter = 0
    for i in range(len(example)) :
        if example.iloc[i , -1] == 1 :
            counter = counter + 1
    if counter >= example.shape[0] / 2:
        return 1 , counter
    return 0 , counter

In [8]:
def entropy (example):
    temp , amount = plurality_value(example)
    probability = amount / example.shape[0]
    if probability == 0 :
        return 0
    if probability == 1 :
        return 0
    return -1 * ( probability * math.log2(probability) + (1 - probability) * math.log2(1 - probability))

test the MakeUnique

In [9]:
MakeUnique(dataset , "Type")

In [10]:
dataset

,ex,Alt,Bar,Fri,Hun,Pat,Price,Rain,Res,Type,Est,Goal
0,x1,Yes,No,No,Yes,Some,$$$,No,Yes,0,"""0 - 10""",1
1,x2,Yes,No,No,Yes,Full,$,No,No,1,"""30 - 60""",0
2,x3,No,Yes,No,No,Some,$,No,No,2,"""0 - 10""",1
3,x4,Yes,No,Yes,Yes,Full,$,Yes,No,1,"""10 - 30""",1
4,x5,Yes,No,Yes,No,Full,$$$,No,Yes,0,"""> 60""",0
5,x6,No,Yes,No,Yes,Some,$$,Yes,Yes,3,"""0 - 10""",1
6,x7,No,Yes,No,No,None,$,Yes,No,2,"""0 - 10""",0
7,x8,No,No,No,Yes,Some,$$,Yes,Yes,1,"""0 - 10""",1
8,x9,No,Yes,Yes,No,Full,$,Yes,No,2,"""> 60""",0
9,x10,Yes,Yes,Yes,Yes,Full,$$$,No,Yes,3,"""10 - 30""",0


In [11]:
type(dataset["Fri"][1])

str

In [12]:
datasetNparr = np.array(dataset)

test the make unique for ints

In [13]:
intlist = [12 , 1 , 123, 23, 22, 5, 77]
dftest = pd.DataFrame(intlist , columns=["test"])


In [14]:
MakeUniqueInt(dftest , "test")

In [15]:
dftest

,test
0,0
1,0
2,6
3,1
4,1
5,0
6,4


test the plurality value

In [16]:
plurality_value(dataset)

(1, 6)

test entropy 

In [17]:
entropy(dataset)

1.0

In [41]:
def Remainder(dataset , feature):
    summation = 0
    temp , p = plurality_value(dataset)
    n = dataset.shape[0] - p
    values = np.unique(dataset[feature])
    # print(values)
    for value in values:
        # print(value)
        temp , counter = plurality_value(dataset.loc[dataset[feature] == value]) 
        pk = counter
        nk = len(dataset.loc[dataset[feature] == value]) - pk
        # print(f"value is{value} and pk is{pk}")
        # print(summation)
        summation = summation + (((pk + nk) / ( p + n)) * entropy(dataset.loc[dataset[feature] == value]))
    return abs(summation)
                

In [19]:
def Gain1(example , feature):
    return entropy(example) - Remainder(example , feature)

In [20]:
dataset.loc[dataset["Goal"] == 1]

,ex,Alt,Bar,Fri,Hun,Pat,Price,Rain,Res,Type,Est,Goal
0,x1,Yes,No,No,Yes,Some,$$$,No,Yes,0,"""0 - 10""",1
2,x3,No,Yes,No,No,Some,$,No,No,2,"""0 - 10""",1
3,x4,Yes,No,Yes,Yes,Full,$,Yes,No,1,"""10 - 30""",1
5,x6,No,Yes,No,Yes,Some,$$,Yes,Yes,3,"""0 - 10""",1
7,x8,No,No,No,Yes,Some,$$,Yes,Yes,1,"""0 - 10""",1
11,x12,Yes,Yes,Yes,Yes,Full,$,No,No,2,"""30 - 60""",1


In [21]:
Remainder(dataset , "Pat")

0.4591479170272448

In [22]:
dataset

,ex,Alt,Bar,Fri,Hun,Pat,Price,Rain,Res,Type,Est,Goal
0,x1,Yes,No,No,Yes,Some,$$$,No,Yes,0,"""0 - 10""",1
1,x2,Yes,No,No,Yes,Full,$,No,No,1,"""30 - 60""",0
2,x3,No,Yes,No,No,Some,$,No,No,2,"""0 - 10""",1
3,x4,Yes,No,Yes,Yes,Full,$,Yes,No,1,"""10 - 30""",1
4,x5,Yes,No,Yes,No,Full,$$$,No,Yes,0,"""> 60""",0
5,x6,No,Yes,No,Yes,Some,$$,Yes,Yes,3,"""0 - 10""",1
6,x7,No,Yes,No,No,None,$,Yes,No,2,"""0 - 10""",0
7,x8,No,No,No,Yes,Some,$$,Yes,Yes,1,"""0 - 10""",1
8,x9,No,Yes,Yes,No,Full,$,Yes,No,2,"""> 60""",0
9,x10,Yes,Yes,Yes,Yes,Full,$$$,No,Yes,3,"""10 - 30""",0


In [23]:
def CompleteUnique(dataset, datasetNparr):
    for i in range (dataset.shape[1]):
        if type(datasetNparr[i]) != str :
            MakeUniqueInt(dataset , dataset.columns[i])
        else :
            MakeUnique(dataset ,  dataset.columns[i])

In [24]:
dataset.head()

,ex,Alt,Bar,Fri,Hun,Pat,Price,Rain,Res,Type,Est,Goal
0,x1,Yes,No,No,Yes,Some,$$$,No,Yes,0,"""0 - 10""",1
1,x2,Yes,No,No,Yes,Full,$,No,No,1,"""30 - 60""",0
2,x3,No,Yes,No,No,Some,$,No,No,2,"""0 - 10""",1
3,x4,Yes,No,Yes,Yes,Full,$,Yes,No,1,"""10 - 30""",1
4,x5,Yes,No,Yes,No,Full,$$$,No,Yes,0,"""> 60""",0


In [25]:
dataset.columns[1]

'Alt'

In [26]:
test = np.array(dataset)

***test the CompleteUnique() function***


In [ ]:
dataset

,ex,Alt,Bar,Fri,Hun,Pat,Price,Rain,Res,Type,Est,Goal
0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,1,1,0,1,1,1,1
2,2,1,1,0,1,0,1,0,1,2,0,0
3,3,0,0,1,0,1,1,1,1,1,2,0
4,4,0,0,1,1,1,0,0,0,0,3,1
5,5,1,1,0,0,0,2,1,0,3,0,0
6,6,1,1,0,1,2,1,1,1,2,0,1
7,7,1,0,0,0,0,2,1,0,1,0,0
8,8,1,1,1,1,1,1,1,1,2,3,1
9,9,0,1,1,0,1,0,0,0,3,2,1


In [42]:
def sameClassification(example):
    temp1 = np.array(example)
    temp = example.iloc[0 , -1]
    for i in range(example.shape[0]):
        if temp != temp1[i , -1]:
            return False
    return True

In [ ]:
test123 = np.array(dataset)
test123[4 , -1]

1

In [ ]:
dataset.iloc[0 , -1]

0

***test the Same Classification***

In [ ]:
sameClassification(dataset)

False

In [43]:
class DecisionTree():
    def __init__(self , node = None):
        self.root = node

In [44]:
def build ( example , features , parent_examples , depth , label = None ):
    testnparr = np.array(example)
    if example.empty:
        return node(value = plurality_value(parent_examples)[0] , type= "leaf" , label = label)
    elif sameClassification(example) :
        return node(value = plurality_value(example)[0] , type= "leaf" , label = label)
    elif len(features) == 0 : 
        return node(value = plurality_value(example)[0] , type= "leaf" , label = label)
    else :
        temp = features[0]
        maxGain = 0 ;
        for i in features :
            if Gain1(example , i) > maxGain :
                maxGain = Gain1(example , i)
                temp = i
        new = node( maxGain , feature= temp , label = label , type="node"  )
        values = np.unique(example[temp])
        features.remove(temp)
        for j in values:
            exs = example[example[temp] == j]
            testexs = np.array(exs)
            # print(j)
            new.children.append(build(exs , features, example ,depth= depth + 1 , label=j))
        return new

In [ ]:
testNode = build(dataset.iloc[: , 1:] , list(dataset.iloc[: , 1:-1].columns) , dataset.iloc[: , 1:] , 0 )

In [45]:
def dfsprint(node , depth):
    if node.type == "leaf":
        print("\t"* depth + f'---leaf---  label = {node.label} ------ value : {node.value} ------ depth : {depth} ---')
    else :
        print("\t"* depth + f'---Node--- --- label : {node.label} ------ gainInfo : {node.gainInfo} ------ depth : {depth} ------ features : {node.feature} ------') 
        for child in node.children:
            dfsprint(child, depth+1)

In [ ]:
testNode

In [ ]:
dfsprint(testNode , 0)

---Node--- --- label : None ------ gainInfo : 0.5408520829727552 ------ depth : 0 ------ features : Pat ------
	---leaf---  label = 0 ------ value : 0 ------ depth : 1 ---
	---Node--- --- label : 1 ------ gainInfo : 0.2516291673878229 ------ depth : 1 ------ features : Hun ------
		---Node--- --- label : 0 ------ gainInfo : 0.5 ------ depth : 2 ------ features : Type ------
			---leaf---  label = 0 ------ value : 1 ------ depth : 3 ---
			---Node--- --- label : 1 ------ gainInfo : 1.0 ------ depth : 3 ------ features : Fri ------
				---leaf---  label = 0 ------ value : 1 ------ depth : 4 ---
				---leaf---  label = 1 ------ value : 0 ------ depth : 4 ---
			---leaf---  label = 2 ------ value : 0 ------ depth : 3 ---
			---leaf---  label = 3 ------ value : 1 ------ depth : 3 ---
		---leaf---  label = 1 ------ value : 1 ------ depth : 2 ---
	---leaf---  label = 2 ------ value : 1 ------ depth : 1 ---


In [46]:
import copy
def predict(node , X):
    testnparr = np.array(X)
    predictions = []
    for i in range(X.shape[0]):
        flag = True
        temp = copy.deepcopy(node)
        while flag :
            for j in temp.children :
                if flag == False : 
                    break
                if X[temp.feature].iloc[i] == j.label :
                    if j.type == "leaf":
                        predictions.append(j.value)
                        flag = False;
                    else:
                        temp = j
                        break
    return  predictions

In [ ]:
dataset["Pat"].iloc[1]

1

In [ ]:
predict(testNode , dataset)

[0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0]

In [ ]:
dataset

,ex,Alt,Bar,Fri,Hun,Pat,Price,Rain,Res,Type,Est,Goal
0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,1,1,0,1,1,1,1
2,2,1,1,0,1,0,1,0,1,2,0,0
3,3,0,0,1,0,1,1,1,1,1,2,0
4,4,0,0,1,1,1,0,0,0,0,3,1
5,5,1,1,0,0,0,2,1,0,3,0,0
6,6,1,1,0,1,2,1,1,1,2,0,1
7,7,1,0,0,0,0,2,1,0,1,0,0
8,8,1,1,1,1,1,1,1,1,2,3,1
9,9,0,1,1,0,1,0,0,0,3,2,1


***Second Part : diabetes***


In [36]:
diabetDataFrame = pd.read_csv("diabetes.csv")
diabetDataFrame

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


***EDA***

In [ ]:
diabetDataFrame.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
diabetDataFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [ ]:
diabetDataFrame.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [37]:
CompleteUnique(diabetDataFrame , np.array(diabetDataFrame))

In [38]:
diabetDataFrame

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,256,568,449,268,0,378.0,126.0,365,384
1,42,326,412,222,0,299.0,62.0,125,0
2,341,702,399,0,0,262.0,136.0,138,384
3,42,341,412,176,85,316.0,20.0,0,0
4,0,526,249,268,152,486.0,507.0,151,384
...,...,...,...,...,...,...,...,...,...
763,426,387,474,368,163,371.0,21.0,528,0
764,85,468,437,207,0,415.0,60.0,75,0
765,213,464,449,176,101,295.0,38.0,113,0
766,42,483,374,0,0,339.0,62.0,327,384


In [ ]:
diabetDataFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [39]:
diabetDataFrame["BMI"] = diabetDataFrame["BMI"].astype(int)
diabetDataFrame["DiabetesPedigreeFunction"] = diabetDataFrame["DiabetesPedigreeFunction"].astype(int)

In [ ]:
diabetDataFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   Pregnancies               768 non-null    int64
 1   Glucose                   768 non-null    int64
 2   BloodPressure             768 non-null    int64
 3   SkinThickness             768 non-null    int64
 4   Insulin                   768 non-null    int64
 5   BMI                       768 non-null    int32
 6   DiabetesPedigreeFunction  768 non-null    int32
 7   Age                       768 non-null    int64
 8   Outcome                   768 non-null    int64
dtypes: int32(2), int64(7)
memory usage: 48.1 KB


In [ ]:
diabetDataFrame

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,256,568,449,268,0,378,126,365,384
1,42,326,412,222,0,299,62,125,0
2,341,702,399,0,0,262,136,138,384
3,42,341,412,176,85,316,20,0,0
4,0,526,249,268,152,486,507,151,384
...,...,...,...,...,...,...,...,...,...
763,426,387,474,368,163,371,21,528,0
764,85,468,437,207,0,415,60,75,0
765,213,464,449,176,101,295,38,113,0
766,42,483,374,0,0,339,62,327,384


***fit***

In [47]:
diabeteTree = build(diabetDataFrame.iloc[: , 1:] , list(diabetDataFrame.iloc[: , 1:-1].columns) , diabetDataFrame.iloc[: , 1:] , 0)

In [48]:
diabeteTree

In [49]:
dfsprint(diabeteTree , 0)

---Node--- --- label : None ------ gainInfo : 0 ------ depth : 0 ------ features : Glucose ------
	---Node--- --- label : 0 ------ gainInfo : 0 ------ depth : 1 ------ features : BloodPressure ------
		---leaf---  label = 299 ------ value : 0 ------ depth : 2 ---
		---Node--- --- label : 424 ------ gainInfo : 0 ------ depth : 2 ------ features : SkinThickness ------
			---leaf---  label = 268 ------ value : 0 ------ depth : 3 ---
			---leaf---  label = 314 ------ value : 0 ------ depth : 3 ---
		---leaf---  label = 462 ------ value : 0 ------ depth : 2 ---
		---leaf---  label = 499 ------ value : 0 ------ depth : 2 ---
	---leaf---  label = 168 ------ value : 0 ------ depth : 1 ---
	---leaf---  label = 215 ------ value : 0 ------ depth : 1 ---
	---leaf---  label = 218 ------ value : 0 ------ depth : 1 ---
	---leaf---  label = 234 ------ value : 0 ------ depth : 1 ---
	---leaf---  label = 238 ------ value : 0 ------ depth : 1 ---
	---leaf---  label = 249 ------ value : 0 ------ depth : 1